In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Data Pre-processing 

In [1]:
# Load data

import pandas as pd
import numpy as np

dataset = pd.read_csv('/kaggle/input/smoking-drinking-dataset/smoking_driking_dataset_Ver01.csv')
dataset.head()

,sex,age,height,weight,waistline,sight_left,sight_right,hear_left,hear_right,SBP,...,LDL_chole,triglyceride,hemoglobin,urine_protein,serum_creatinine,SGOT_AST,SGOT_ALT,gamma_GTP,SMK_stat_type_cd,DRK_YN
0,Male,35,170,75,90.0,1.0,1.0,1.0,1.0,120.0,...,126.0,92.0,17.1,1.0,1.0,21.0,35.0,40.0,1.0,Y
1,Male,30,180,80,89.0,0.9,1.2,1.0,1.0,130.0,...,148.0,121.0,15.8,1.0,0.9,20.0,36.0,27.0,3.0,N
2,Male,40,165,75,91.0,1.2,1.5,1.0,1.0,120.0,...,74.0,104.0,15.8,1.0,0.9,47.0,32.0,68.0,1.0,N
3,Male,50,175,80,91.0,1.5,1.2,1.0,1.0,145.0,...,104.0,106.0,17.6,1.0,1.1,29.0,34.0,18.0,1.0,N
4,Male,50,165,60,80.0,1.0,1.2,1.0,1.0,138.0,...,117.0,104.0,13.8,1.0,0.8,19.0,12.0,25.0,1.0,N


In [2]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 991346 entries, 0 to 991345
Data columns (total 24 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   sex               991346 non-null  object 
 1   age               991346 non-null  int64  
 2   height            991346 non-null  int64  
 3   weight            991346 non-null  int64  
 4   waistline         991346 non-null  float64
 5   sight_left        991346 non-null  float64
 6   sight_right       991346 non-null  float64
 7   hear_left         991346 non-null  float64
 8   hear_right        991346 non-null  float64
 9   SBP               991346 non-null  float64
 10  DBP               991346 non-null  float64
 11  BLDS              991346 non-null  float64
 12  tot_chole         991346 non-null  float64
 13  HDL_chole         991346 non-null  float64
 14  LDL_chole         991346 non-null  float64
 15  triglyceride      991346 non-null  float64
 16  hemoglobin        99

In [4]:
dataset['sex'].value_counts()

sex
Male      526415
Female    464931
Name: count, dtype: int64

In [5]:
# Encoding 'sex' to 0, 1 

dataset.replace({'sex': {'Male': 0, 'Female': 1}}, inplace = True)
dataset['sex'].value_counts()

sex
0    526415
1    464931
Name: count, dtype: int64

In [6]:
# Encoding 'DRK_YN' to 0, 1 

dataset.replace({'DRK_YN':{'Y':1, 'N':0}}, inplace = True)
dataset['DRK_YN'].value_counts()

DRK_YN
0    495858
1    495488
Name: count, dtype: int64

In [7]:
dataset.groupby('DRK_YN').mean()

,sex,age,height,weight,waistline,sight_left,sight_right,hear_left,hear_right,SBP,...,HDL_chole,LDL_chole,triglyceride,hemoglobin,urine_protein,serum_creatinine,SGOT_AST,SGOT_ALT,gamma_GTP,SMK_stat_type_cd
DRK_YN,,,,,,,,,,,,,,,,,,,,,
0,0.652929,51.648809,158.764848,59.977998,80.131769,0.933964,0.933205,1.041679,1.040481,121.950591,...,56.204200,114.603191,121.476582,13.755567,1.095592,0.831262,25.064444,24.100658,26.798521,1.321110
1,0.284913,43.577160,165.718998,66.592571,82.335770,1.027740,1.023687,1.021302,1.020463,122.914765,...,57.669948,111.471025,142.814884,14.704434,1.092856,0.889693,26.914864,27.410680,47.481893,1.895348


In [14]:
# Data Split
from sklearn.model_selection import train_test_split

X = dataset.drop(columns = ['DRK_YN'], axis = 1)
Y = dataset['DRK_YN']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y, random_state = 1)
print(X.shape, X_train.shape, X_test.shape)

(991346, 23) (793076, 23) (198270, 23)


# Model Training

In [15]:
# Model selection

from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver = 'sag')


In [16]:
model.fit(X_train, Y_train)

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


LogisticRegression(solver='sag')

In [17]:
# Performance evaluation
from sklearn.metrics import accuracy_score

Y_pred = model.predict(X_test)
acc = accuracy_score(Y_test, Y_pred)
print(acc)

0.7054622484490846
